### **STEP-1**. 준비 작업
AI contents 실행 (python main.py) 후에 등록 과정을 진행 할 수 있습니다. !! 

jupyter kernel 을 제작합니다. 
```bash
    conda activate {ENV-NAME}           ## python main.py 실행 용 가상환경
    pip install ipykernel        
    python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}
```


----

### **STEP-2**. Solution 정보 입력 및 Infra 정보 입력

##### Solution information (dict) 작성 
- inference_only: 파이프라인이 학습 및 추론 과정을 지원하는 표시     
  - ex1: 학습 과 추론 모두 지원 --> False     
  - ex2: 추론만 지원 --> True    
  - ex3: 학습 만은 미지원 --> ~~['train']~~ 

- solution_update: 기존 솔루션을 업데이트 할지 결정
  - True: 업데이트 진행. 기존 name 과 동일한 이름 입력 (name 이 존재하지 않으면 에러)  
  - False: 신규 생성. 기존 name 과 다르게 입력 (name 이 존재하면 에러)

- solution_name: 솔루션 이름 
  - 주의사항: 스페이스, 특수 문자, 한글은 미지원 

- solution_type: AI Solution 이 공유되는 계정 범위. 'private', 'public' 중에 하나 선택.   
  - 'public' : AI-Advisor 가 설치될 때, built-in 되어 설치 할 solution 의미.   
    (SYSTEM MANAGER 만 등록 가능)
  - 'private': 과제 단위로 공유 될 solution. 

- description: 솔루션이 Edge Conductor 의 UI 로 표시 될 때, 설명 가이드 작성 
  - overview: 솔루션에 대한 전반적을 설명을 작성. (markdown 을 지원)
  - 고려사항: overview 외에 다양한 sub-title 지원 예정.

- contents_type: re-train, re-labeling 을 위한 설명 
  - support_labeling: re-labeling 할지 결정. True 일 경우,   
    Edge Conductor 에서 re-labeling 기능 활성화.   
  - inference_result_datatype: EdgeApp 에 inference 결과를 표시하는 방법으로 'table', 'image' 중에 하나 선택. 
    AI Contents 제작 시, output.csv, output.jpg 를 생성해 두어야 함
  - train_datatype: re-train 에 사용될 데이터 포멧 결정으로 'table', 'image' 중에 하나 선택. 
    AI Contents 제작 시, output.csv, output.jpg 를 생성해 두어야 함

- train_gpu: True, False 중에 선택. True 일 경우, gpu 용 train docker container 제작 
- train_gpu: True, False 중에 선택. True 일 경우, gpu 용 inference docker container 제작 
- inference_arm: True, False 중에 선택. True 일 경우, arm 용 inference docker container 제작


    

In [1]:
#----------------------------------------#
#        AI Solution Spec 작성           #
#----------------------------------------#
solution_info ={
    'inference_only': False, # True, False
    'solution_update': True,
    'solution_name': 'tcr-test-ssh-v6',
    'solution_type': "private", ##public, private 
    'description': {
        "overview": "Provide a description of the AI solution."
    },
    'contents_type': {
            'support_labeling': False,
            'inference_result_datatype': 'table', # 'image'
            'train_datatype': 'table' # 'image'
    },
    'train_gpu': False, ## cpu, gpu
    'inference_gpu': False,
    "inference_arm": False,  # amd, arm  
}

##### AI Solution 이 동작될 Infra_setup 파일 load 하기
./setting/infra_setup.yaml 를 참조하여 infra 환경을 설정합니다. 

### **STEP-3**. AI Solution 등록

##### 3-1. Username & Password 를 입력 

> !! 접속이 실패되는 경우, AI Conductor 에게 계정 생성을 요청 하시기 바랍니다.   
> Contact Us: hyunsoo0802.lim@lge.com  (LGE AI빅데이터담당, AI Conductor 담당자)

> Login 방식이 {'LOGIN_METHOD': 'ldap'} 로 설정되어 있는 경우, LGEP 의 ID & PW 를 사용


In [2]:
import getpass

username = input('Username: ')
password = getpass.getpass('Password: ')

print("Your ID : ", username)
print("Your PW : ", password.replace(password, '*' * len(password)))

Your ID :  cism-mgr
Your PW :  ************


##### 3-2. ALO 솔루션 등록 실행
아래와 같은 과정이 순차적으로 실행 됨. 진행 중 Error 가 발생하게 되면, STEP-2 설정을 변경 함.
> Contact US: sehyun.song@lge.com (LGE AI빅데이터 담당, ALO 담당자)

AI Solution 등록 과정 Process:  
> 1.  solution name 입력 
> 2.  description & wrangler code 를 solution_metadata 에 삽입
> 3.  contents_type 을 solution_metadata 에 삽입
> 4.  description 의 solution icon 을 solution_metadata 에 삽입
> 5.  train 용 user parameter 를 solution_metadata 에 삽입
> 6.  s3 에 train data & artifacts 업로드 
> 7.  train 용 ecr repository 생성 및 docker container 업로드 
> 8.  inference 용 user parameter 를 solution_metadata 에 삽입
> 9.  s3 에 inference data & artifacts 업로드 
> 10. inference 용 ecr repository 생성 및 docker container 업로드 
> 11. solution 등록 



In [3]:
import sys

try:
    del sys.modules['src.solution_register'] 
    del sys.modules['src.contants'] 
except:
    pass
from src.solution_register import SolutionRegister

# infra = "./setting/infra_config.yaml"
# infra = "./setting/example_infra_config/infra_config.meerkat.yaml"
infra = "./setting/example_infra_config/infra_config.localtest.yaml"
register = SolutionRegister(infra_setup=infra, solution_info=solution_info)

register.login(username, password)


######################################################
#######    Initiate ALO operation mode
########################################################

[SYSTEM] Solutoin 등록에 필요한 setup file 들을 load 합니다. 
Infra setup 파일을 load 합니다. (path: ./setting/example_infra_config/infra_config.localtest.yaml)
[SYSTEM] infra_setup (max display: 5 line): 
{'AIC_URI': 'https://web.aic-dev.lgebigdata.com/',
 'AWS_KEY_FILE': '../aws.key',
 'BUILD_METHOD': 'docker',
 'ECR_TAG': 'latest',
 'LOGIN_MODE': 'static',
 'REGION': 'ap-northeast-2',
 'REPOSITORY_TAGS': ['Key=Company,Value=LGE',
                     'Key=Owner,Value=IC360',
                     'Key=HQ,Value=CDO',
                     'Key=Division,Value=CDO',
                     'Key=Infra Region,Value=KIC',
                     'Key=Service Mode,Value=OP',
                     'Key=Cost Type,Value=COMPUTING',
                     'Key=Project,Value=CIS',
                     'Key=Sub Project,Value=CISM',
                     'Key=System,Value=AI

In [4]:
register.debugging = False  ## default: False (skip 항목: docker 생성, solution 등록)
register.skip_generation_docker = False  ## default: False
register.run(username=username, password=password)


######################################################
#######    Login to AI Conductor
########################################################

[SUCCESS] Login 접속을 성공하였습니다. 
[INFO] Login response: 
 {'account_id': '0b3d6142-1011-4adc-bbf8-ff5cc4f59883', 'display_name': 'CISM Manager Account', 'role': 'DEVELOPER', 'workspace': [{'id': '1b61dae3-e8f7-4b5f-9629-da0aedbb4a08', 'name': 'cism-ws', 'role': 'MANAGER'}]}
"해당 계정으로 접근 가능한 workspace list: ['cism-ws']"
[SYSTEM] 접근 요청하신 workspace (cism-ws) 은 해당 계정으로 접근 가능합니다.

######################################################
#######    Solution Name Creation
########################################################

<Response [200]>
[SUCCESS] The Solution Name you entered (tcr-test-ssh-v6) already exists and can be upgraded. 
[SYSTEM] Solution Name List (in-use):
╒════╤══════════════════════════╕
│    │ AI solutions             │
╞════╪══════════════════════════╡
│  0 │ version-test-cism-2      │
├────┼──────────────────────────┤
│  1 │ tcr-

WARNING! Your password will be stored unencrypted in /home/sehyun.song/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



[SYSTEM] AWS ECR | docker login result:
Login Succeeded

[SYSTEM] Target AWS ECR repository:
ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh-v6/train/tcr-test-ssh-v6
[SYSTEM] AWS ECR create-repository response: 
{'repository': {'repositoryArn': 'arn:aws:ecr:ap-northeast-2:086558720570:repository/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh-v6/train/tcr-test-ssh-v6', 'registryId': '086558720570', 'repositoryName': 'ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh-v6/train/tcr-test-ssh-v6', 'repositoryUri': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh-v6/train/tcr-test-ssh-v6', 'createdAt': datetime.datetime(2024, 2, 2, 18, 3, 39, 725000, tzinfo=tzlocal()), 'imageTagMutability': 'MUTABLE', 'imageScanningConfiguration': {'scanOnPush': False}, 'encryptionConfiguration': {'encryptionType': 'AES256'}}, 'ResponseMetadata': {'RequestId': '10bdd479-9d68-4808-b30c-165a07579e60', 'HTTPStatusCode': 200, 'HTTPHe

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 1.21kB done
#1 DONE 0.2s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.3s

#3 [internal] load metadata for public.ecr.aws/docker/library/python:3.10-slim-bullseye
#3 DONE 0.0s

#4 [ 1/10] FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 241.54kB 0.0s done
#5 DONE 0.2s

#6 [ 3/10] RUN apt-get install -y apt-utils
#6 CACHED

#7 [ 2/10] RUN apt-get update
#7 CACHED

#8 [ 4/10] RUN apt-get install -y --no-install-recommends          build-essential          wget          ca-certificates          git          gcc     && rm -rf /var/lib/apt/lists/*
#8 CACHED

#9 [ 5/10] COPY /.register_source /framework
#9 DONE 1.8s

#10 [ 6/10] WORKDIR /framework
#10 DONE 0.7s

#11 [ 7/10] RUN pip3 install --upgrade pip
#11 2.485 Requirement already satisfied: pip in /usr/local/lib/python3.10/site-packages (23.0.1)


The push refers to repository [086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh-v6/train/tcr-test-ssh-v6]
5093fc227810: Preparing
7ed7afe052b2: Preparing
6d8cdb4ec8ae: Preparing
13142114a1d8: Preparing
5f70bf18a086: Preparing
5177e69ebb94: Preparing
e60348d68089: Preparing
4cac178e7314: Preparing
dc4295a3639b: Preparing
9647f452a529: Preparing
7972dfbb0bc8: Preparing
29578fbbec63: Preparing
cc056986b725: Preparing
74c0af6e0227: Preparing
9647f452a529: Waiting
7972dfbb0bc8: Waiting
29578fbbec63: Waiting
5177e69ebb94: Waiting
e60348d68089: Waiting
4cac178e7314: Waiting
cc056986b725: Waiting
dc4295a3639b: Waiting
74c0af6e0227: Waiting
5f70bf18a086: Pushed
6d8cdb4ec8ae: Pushed
13142114a1d8: Pushed
5177e69ebb94: Pushed
7ed7afe052b2: Pushed
4cac178e7314: Pushed
7972dfbb0bc8: Pushed
9647f452a529: Pushed
cc056986b725: Pushed
dc4295a3639b: Pushed
29578fbbec63: Pushed
74c0af6e0227: Pushed
e60348d68089: Pushed
5093fc227810: Pushed
v2: diges

WARNING! Your password will be stored unencrypted in /home/sehyun.song/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



[SYSTEM] AWS ECR | docker login result:
Login Succeeded

[SYSTEM] Target AWS ECR repository:
ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh-v6/inference/tcr-test-ssh-v6
[SYSTEM] AWS ECR create-repository response: 
{'repository': {'repositoryArn': 'arn:aws:ecr:ap-northeast-2:086558720570:repository/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh-v6/inference/tcr-test-ssh-v6', 'registryId': '086558720570', 'repositoryName': 'ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh-v6/inference/tcr-test-ssh-v6', 'repositoryUri': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh-v6/inference/tcr-test-ssh-v6', 'createdAt': datetime.datetime(2024, 2, 2, 18, 8, 54, 813000, tzinfo=tzlocal()), 'imageTagMutability': 'MUTABLE', 'imageScanningConfiguration': {'scanOnPush': False}, 'encryptionConfiguration': {'encryptionType': 'AES256'}}, 'ResponseMetadata': {'RequestId': '28f60a16-1775-46c6-a965-0651aa16a6e3', 'HTTPStatusCod

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 1.66kB done
#1 DONE 0.3s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.2s

#3 [internal] load metadata for public.ecr.aws/docker/library/python:3.10-slim-bullseye
#3 DONE 0.0s

#4 [ 1/12] FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 116.63kB 0.0s done
#5 DONE 0.2s

#6 [ 2/12] RUN apt-get update
#6 CACHED

#7 [ 3/12] RUN apt-get install -y apt-utils
#7 CACHED

#8 [ 4/12] RUN apt-get install -y --no-install-recommends          build-essential          wget          ca-certificates          git          gcc          docker.io          procps          jq     && rm -rf /var/lib/apt/lists/*
#8 CACHED

#9 [ 5/12] COPY /.register_source /framework/
#9 DONE 1.5s

#10 [ 6/12] COPY /solution_metadata.yaml /framework/
#10 DONE 0.9s

#11 [ 7/12] RUN mkdir -p /alo/artifacts/inference_artifacts/log &&    

The push refers to repository [086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh-v6/inference/tcr-test-ssh-v6]
9d4803ed783e: Preparing
a37fdb9958fa: Preparing
2e1b93999779: Preparing
7abd7512d66e: Preparing
5f70bf18a086: Preparing
a88a9f0ce8a4: Preparing
7278c2769df6: Preparing
20150d7059d1: Preparing
0941f5db1f5c: Preparing
4cac178e7314: Preparing
dc4295a3639b: Preparing
9647f452a529: Preparing
7972dfbb0bc8: Preparing
29578fbbec63: Preparing
cc056986b725: Preparing
74c0af6e0227: Preparing
a88a9f0ce8a4: Waiting
7278c2769df6: Waiting
20150d7059d1: Waiting
dc4295a3639b: Waiting
0941f5db1f5c: Waiting
29578fbbec63: Waiting
4cac178e7314: Waiting
9647f452a529: Waiting
7972dfbb0bc8: Waiting
cc056986b725: Waiting
74c0af6e0227: Waiting
5f70bf18a086: Pushed
2e1b93999779: Pushed
7abd7512d66e: Pushed
a88a9f0ce8a4: Pushed
7278c2769df6: Pushed
20150d7059d1: Pushed
4cac178e7314: Pushed
a37fdb9958fa: Pushed
7972dfbb0bc8: Pushed
9647f452a529: Push

##### 등록된 AI Solution 테스트 하기. (Manager 권한만 가능)
- 테스트에 사용된 stream, solution instance 를 삭제 가능

In [6]:
register.run_train(
    status_period = 10, ## 몇 초 간격으로 학습 상태를 체크할 것인지 설정
    delete_instance = False,   ## 학습 테스트에 사용한 solution instance & stream 을 삭제할 것인지 설정 
    delete_solution= False,  ## solution 을 삭제할 것인지 설정
    )


######################################################
#######    Register AI solution instance
########################################################

[SYSTEM] AI solution 등록 정보를 /home/sehyun.song/Project/alo/dev-240130/.register_interface/.response_solution.json 에서 확인합니다.

[INFO] AI solution interface information: 
 {'workspace_name': 'cism-ws'}
[ERROR] AI solution instance 등록을 실패하였습니다. 잘못된 요청입니다. 
Error message:  app.handlers.instances_v1_0 : Failed to create Instance, already exist name alo-test-tcr-test-ssh-v6

######################################################
#######    Register AI solution stream
########################################################



ValueError: [ERROR] /home/sehyun.song/Project/alo/dev-240130/.register_interface/.response_solution_instance.json 를 읽기 실패 하였습니다.

##### 등록한 Solution List 를 확인하기 (Manager 권한만 가능)

In [5]:
register.list_solution()


######################################################
#######    Load AI solution instance list
########################################################


[INFO] AI solution interface information: 
 {'workspace_name': 'cism-ws', 'with_pulic': 1, 'page_size': 100}
[SUCCESS] solution list 조회를 성공하였습니다. 
'[INFO] response: '
(idx: 0), solution_name: version-test-cism-2, solution_id: 2a2a1121-de8f-49ae-8961-93f904da1252, latest_version: 2
(idx: 1), solution_name: tcr-release-v1.2-test1, solution_id: b1d9f4e8-b210-4c82-a275-01b901c6d2ac, latest_version: 1
(idx: 2), solution_name: tcr-release-v.1.2-test1, solution_id: 4c5d169e-9863-4590-b8b0-9986a7231c7b, latest_version: 1
(idx: 3), solution_name: titanic-test-v1, solution_id: 2b136994-f4c4-4b2c-bd93-daa000e199ea, latest_version: 1
(idx: 4), solution_name: tcr-test-ssh-v6, solution_id: eb5eec8a-78d7-4c7a-aa32-90338129758e, latest_version: 2
(idx: 5), solution_name: tcr-test-swj-0129, solution_id: a94f3b97-fc9e-44fb-a51a-11a5239752cd, latest_

##### Solution 을 삭제 하기

In [ ]:
register.delete_solution()